# INCEPTION-V3 NEURAL NETWORK MODEL - TENSORFLOW IMPLEMENTATION

## Imports

***mount drive***

In [1]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


***general imports***

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os

***tensorflow imports***

In [3]:
import tensorflow as tf 
from tensorflow import keras

## Load Data

***types of data***

In [4]:
# the path to the skin cancer different types data
dataset_path = "/content/drive/MyDrive/Youtube/5 - inception-v3/dataset/"
                          
# the different types of skin cancer
cancer_types = os.listdir(dataset_path)

# the number of different types of skin cancer
print("Types of cancer found: ", len(cancer_types))

Types of cancer found:  3


***loop over dataset***

In [5]:
import cv2

im_size = 299

images = []
labels = []

for i in cancer_types:
    data_path = dataset_path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
    print(filenames)  
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)  
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

labels = np.array(labels)
labels.shape

['ISIC_0000151.jpg', 'ISIC_0000155 - Copy.jpg', 'ISIC_0000152 - Copy.jpg', 'ISIC_0000157.jpg', 'ISIC_0000152.jpg', 'ISIC_0000161.jpg', 'ISIC_0000160.jpg', 'ISIC_0000154 - Copy.jpg', 'ISIC_0000155.jpg', 'ISIC_0000156 - Copy.jpg', 'ISIC_0000159.jpg', 'ISIC_0000162.jpg', 'ISIC_0000153 - Copy.jpg', 'ISIC_0000153.jpg', 'ISIC_0000156.jpg', 'ISIC_0000154.jpg', 'ISIC_0000158.jpg', 'ISIC_0000440.jpg', 'ISIC_0000434 - Copy.jpg', 'ISIC_0000440 - Copy.jpg', 'ISIC_0000444 - Copy.jpg', 'ISIC_0000441 - Copy.jpg', 'ISIC_0000444.jpg', 'ISIC_0000519 - Copy.jpg', 'ISIC_0000521 - Copy.jpg', 'ISIC_0000484 - Copy.jpg', 'ISIC_0000469 - Copy.jpg', 'ISIC_0000463 - Copy.jpg', 'ISIC_0000522.jpg', 'ISIC_0000466.jpg', 'ISIC_0000533.jpg', 'ISIC_0000459 - Copy.jpg', 'ISIC_0000516 - Copy.jpg', 'ISIC_0000462 - Copy.jpg', 'ISIC_0000469.jpg', 'ISIC_0000547.jpg', 'ISIC_0000445.jpg', 'ISIC_0000516.jpg', 'ISIC_0000511 - Copy.jpg', 'ISIC_0000502.jpg', 'ISIC_0000487 - Copy.jpg', 'ISIC_0000466 - Copy.jpg', 'ISIC_0000522 - Cop

(178,)

## Pre-Process Data

***convert to numpy array***

In [6]:
labels = np.array(labels)
images = np.array(images)

images.shape   

(178, 299, 299, 3)

***normalize data***

In [7]:
images = images.astype('float32') / 255.0

***one-hot encode labels***

In [8]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

labelencoder = LabelEncoder ()
labels = labelencoder.fit_transform (labels)

print (labels)

labels = labels.reshape(-1,1)

onehotencoder=OneHotEncoder(categories='auto', sparse=False)

labels = onehotencoder.fit_transform(labels)
labels.shape 

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


(178, 3)

***shuffle and train/test split***

In [9]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, labels = shuffle(images, labels, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=415)

#inpect the shape of the training and testing.
print(X_train.shape)
print(y_train.shape)

(142, 299, 299, 3)
(142, 3)


## Build Model Architecture

***some useful constants***

In [10]:
# determining the input shape 
img_input = keras.Input(shape=(299, 299, 3))

# number of classes 
classes = 3

# RGB color channels
channel_axis = 3

***convolutional block (conv2d + batch normalization + activation)***

In [11]:
def conv2d_bn(x, filters, num_row, num_col, padding="same", strides=(1, 1)):

  # adding conv-2d layer 
  x = keras.layers.Conv2D(filters, (num_row, num_col),strides=strides,padding=padding)(x)
    
  # adding batch normalization 
  x = keras.layers.BatchNormalization(axis=3, scale=False)(x)
    
  # adding activation function 
  x = keras.layers.Activation("relu")(x)

  return x

***inception-A block***

In [12]:
def inc_block_a(x):    
  
  branch1x1 = conv2d_bn(x, 64, 1, 1)  

  branch5x5 = conv2d_bn(x, 48, 1, 1)  
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

  branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = keras.layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis)

  return x

***reduction-A block***

In [13]:
def reduction_block_a(x):  

  branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding="valid")

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding="valid")

  branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
  x = keras.layers.concatenate([branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)

  return x

***inception-B block***

In [14]:
def inc_block_b(x):

    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1),padding="same")(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = keras.layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool], axis=channel_axis)

    return x

***reduction-B block***

In [15]:
def reduction_block_b(x): 

    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,strides=(2, 2), padding="valid")

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn( branch7x7x3, 192, 3, 3, strides=(2, 2), padding="valid")

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis)

    return x

***inception-C block***

In [16]:
def inc_block_c(x):        

        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = keras.layers.concatenate([branch3x3_1, branch3x3_2],axis=channel_axis)

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = keras.layers.concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding="same")(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = keras.layers.concatenate( [branch1x1, branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)

        return x

***building full model architecture using above functions (converting input to output)***

In [17]:
x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding="valid") 
x = conv2d_bn(x, 32, 3, 3, padding="valid")  
x = conv2d_bn(x, 64, 3, 3) 

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)   
x = conv2d_bn(x, 80, 1, 1, padding="valid") 
x = conv2d_bn(x, 192, 3, 3, padding="valid")  
x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)  

x=inc_block_a(x) 
x=inc_block_a(x) 
x=inc_block_a(x) 

x=reduction_block_a(x) 

x=inc_block_b(x) 
x=inc_block_b(x) 
x=inc_block_b(x) 
x=inc_block_b(x) 

x=reduction_block_b(x) 

x=inc_block_c(x) 
x=inc_block_c(x) 

x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x) 
x = keras.layers.Dense(classes, activation="softmax", name="predictions")(x) 

***define the model using the input and output***

In [18]:
inputs = img_input
model =  keras.Model(inputs, x, name="inception_v3")
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [19]:
from tensorflow.keras.utils import  plot_model
plot_model(model, show_shapes=True, to_file="Inception_model_3.png")

Output hidden; open in https://colab.research.google.com to view.

## Compile and Train Model

***compile model***

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

***train model***

In [26]:
model.fit(X_train, y_train, epochs = 7, batch_size = 16) 

Epoch 1/7
9/9 [==============================] - 50s 232ms/step - loss: 1.1710 - accuracy: 0.5634
Epoch 2/7
9/9 [==============================] - 2s 225ms/step - loss: 0.8034 - accuracy: 0.6761
Epoch 3/7
9/9 [==============================] - 2s 227ms/step - loss: 0.8025 - accuracy: 0.5563
Epoch 4/7
9/9 [==============================] - 2s 227ms/step - loss: 0.7285 - accuracy: 0.6549
Epoch 5/7
9/9 [==============================] - 2s 230ms/step - loss: 0.6361 - accuracy: 0.6620
Epoch 6/7
9/9 [==============================] - 2s 234ms/step - loss: 0.5266 - accuracy: 0.7324
Epoch 7/7
9/9 [==============================] - 2s 229ms/step - loss: 0.6222 - accuracy: 0.7254


## Evaluate Model Performance

***test model***

In [27]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 2s 46ms/step - loss: 3.4174 - accuracy: 0.5556


[3.417403221130371, 0.5555555820465088]